In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
import cv2
from tqdm import tqdm_notebook
from random import shuffle, seed
import shutil
import pandas as pd
import glob

In [ ]:
def organize_datasets(path_to_data, n=38, ratio=0.2):
    files_types = ['depth', 'ir1', 'ir2', 'rgb']
    seed(10)
    subjects = ['{:02d}'.format(i) for i in range(1,n)]
    classes = ['{:02d}'.format(c) for c in range(1,20)]
    #print(classes)
    
    shutil.rmtree(os.path.join(path_to_data,'data'))
    print(os.path.join(path_to_data,'data'), 'removed')
        
    shuffle(subjects)
    #print(subjects)
    
    n = int(len(subjects) * ratio)
    val, train = subjects[:n], subjects[n:]
        
    for file_type in files_types:      
        
        for c in classes: 
            os.makedirs(os.path.join(path_to_data,'data',file_type,'train',c))
            os.makedirs(os.path.join(path_to_data,'data',file_type,'validation',c))

            print('folders created !')

            for t in tqdm_notebook(train):
                files_list = glob.glob(path_to_data + '\\all\\' + c + '\\' + t + '*' + file_type + '*')
                print(path_to_data + '\\all\\' + c + '\\' + t + '*' + file_type + '*', len(files_list))
                for f in files_list:
                    shutil.copy2(f, os.path.join(path_to_data, 'data', file_type, 'train', c))

            for v in tqdm_notebook(val):
                files_list = glob.glob(path_to_data + '\\all\\' + c + '\\' + v + '*' + file_type + '*')
                print(path_to_data + '\\all\\' + c + '\\' + v + '*' + file_type + '*', len(files_list))
                for f in files_list:
                    shutil.copy2(f, os.path.join(path_to_data, 'data', file_type, 'validation', c))

    print('Data copied!')

In [ ]:
organize_datasets(path_to_data='E:\\MestradoData\\preprocessed', n=38, ratio=0.2)

In [43]:
# dimensions of our images.
img_width, img_height = 124, 124

In [28]:
train_data_dir ='C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb\\validation'
nb_train_samples = 15788
nb_validation_samples = 3024
nb_epoch = 50

In [44]:
input_img = Input(shape=(img_width,img_height, 3))  # adapt this if using `channels_first` image data format

In [45]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

In [46]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [47]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [48]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 124, 124, 3)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 124, 124, 32)      896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 31, 31, 64)        36928     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 64)        0         
__________

In [16]:
batch_size = 100

In [49]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.3,
        width_shift_range=0.4,
        height_shift_range=0.4,
        zca_whitening=True,
        zca_epsilon=1e-6,
        rotation_range=0.,
        channel_shift_range=0.2,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=False,
        vertical_flip=False,
        preprocessing_function=None)
    
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [50]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='input',
        #color_mode='grayscale'
        save_to_dir='C:\\Users\\rafae\\Desktop\\Coleta\\data\\auto\\train\\',
        save_prefix='augmented_',
        shuffle=True,
        seed=9876
        )

print(type(train_generator))

Found 15788 images belonging to 19 classes.
<class 'keras.preprocessing.image.DirectoryIterator'>


In [51]:
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='input',
        #color_mode='grayscale')
        save_to_dir='C:\\Users\\rafae\\Desktop\\Coleta\\data\\auto\\validation\\',
        save_prefix='augmented_',
        shuffle=True,
        seed=9876
        )

Found 3024 images belonging to 19 classes.


In [52]:
import keras
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [ ]:
fitted_model = autoencoder.fit_generator(
        train_generator,
        steps_per_epoch= 60000,
        epochs=15,
        validation_data=validation_generator,
        validation_steps= nb_validation_samples,
        callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True), early_stopping, history],
        verbose=0)

HBox(children=(IntProgress(value=0, description='Training', max=15), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=60000), HTML(value='')))

C:\Users\rafae\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py:556: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
autoencoder.save('autoencoder.h5')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.3,
        height_shift_range=0.3)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

print(type(train_generator))

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(19, activation='softmax'))

In [ ]:
epochs = 50
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
fitted_model = model.fit_generator(
        train_generator,
        steps_per_epoch= nb_train_samples,
        epochs=50,
        validation_data=validation_generator,
        validation_steps= nb_validation_samples,
        callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True), early_stopping, history],
        verbose=0)

In [ ]:
model.save('model1.h5')

In [ ]:
losses, val_losses = history.losses, history.val_losses
fig = plt.figure(figsize=(15, 5))
plt.plot(fitted_model.history['loss'], 'g', label="train losses")
plt.plot(fitted_model.history['val_loss'], 'r', label="val losses")
plt.grid(True)
plt.title('Training loss vs. Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
losses, val_losses = history.losses, history.val_losses
fig = plt.figure(figsize=(15, 5))
plt.plot(fitted_model.history['acc'], 'g', label="accuracy on train set")
plt.plot(fitted_model.history['val_acc'], 'r', label="accuracy on validation set")
plt.grid(True)
plt.title('Training Accuracy vs. Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()